# Введение в обработку естественного языка
## Урок 2. Создание признакового пространства

In [1]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=3727f9497cab54146ff09c7e906783002619ae30e46b61f1a9384d68ed5bba09
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
import nltk
import re
import wget
from datetime import datetime
from bs4 import BeautifulSoup
import requests

In [4]:
!unzip "/content/Lesson-2.zip" -d "/content/"

Archive:  /content/Lesson-2.zip
   creating: /content/image/
  inflating: /content/image/2D_tsne.PNG  
  inflating: /content/image/CBOW_.png  
  inflating: /content/image/skipgram.png  
  inflating: /content/image/tf_idf.PNG  
  inflating: /content/image/w2v_formula.PNG  
  inflating: /content/image/word_embeddings.PNG  
  inflating: /content/corpus         
  inflating: /content/lesson_2.ipynb  
  inflating: /content/sem2.ipynb     


In [5]:
HEADER = {
    "User-Agent": 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'
}

url = 'https://gbcdn.mrgcdn.ru/uploads/asset/2800511/attachment/aa4f455409c2e805aa4b13c4011764fa'

response = requests.get(url, headers=HEADER)

with open("d_corpus", "w") as f:
    f.write(response.text)

In [6]:
import filecmp

filecmp.cmp('/content/corpus', '/content/d_corpus')

True

In [7]:
with open('corpus', 'r') as f:
    df = pd.DataFrame({'category': t[0], 'text': t[1]} for t in (l.split(' ', 1) for l in f.readlines()))
df['category'] = df['category'].apply(lambda t: re.search(r'\d+', t).group())
df

,category,text
0,2,Stuning even for the non-gamer: This sound tra...
1,2,The best soundtrack ever to anything.: I'm rea...
2,2,Amazing!: This soundtrack is my favorite music...
3,2,Excellent Soundtrack: I truly like this soundt...
4,2,"Remember, Pull Your Jaw Off The Floor After He..."
...,...,...
9995,2,A revelation of life in small town America in ...
9996,2,Great biography of a very interesting journali...
9997,1,Interesting Subject; Poor Presentation: You'd ...
9998,1,Don't buy: The box looked used and it is obvio...


In [8]:
x_train, x_test, y_train, y_test = train_test_split(df.text, df.category)

In [9]:
from string import punctuation
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from nltk import ngrams
import nltk
from nltk import collocations 
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords
nltk.download('genesis')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package genesis to /root/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [10]:
noise = stopwords.words('english') + list(punctuation)
tokens = [token for sense in df.text for token in word_tokenize(sense) if token not in punctuation]

In [11]:
freq_dict = Counter(tokens)
freq_dict_sorted= sorted(freq_dict.items(), key=lambda x: -x[1])
max([el[1] for el in list(freq_dict_sorted)]), min([el[1] for el in list(freq_dict_sorted)])

(35041, 1)

### Задание 1.
Задание: обучите три классификатора:

1) на токенах с высокой частотой

2) на токенах со средней частотой

3) на токенах с низкой частотой

In [12]:
high_tokens = set()
med_tokens = set()
low_tokens = set()
h = 20000
l = 5000
for i in freq_dict_sorted:
    if i[1] > h:
        high_tokens.add(i[0])
    elif i[1] < l:
        low_tokens.add(i[0])
    else:
        med_tokens.add(i[0])
print(len(high_tokens),len(med_tokens),len(low_tokens))

3 19 47469


In [13]:
sw = noise + list(low_tokens) + list(med_tokens)

In [14]:
vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'`living", "'accelerator", "'adoption", "'anno", "'arular", "'ascension", "'ashes", "'away", "'bertha", "'big", "'brave", "'bubbabrain", "'bubbabyte", "'bucky", "'buffy", "'cade", "'canon", "'catch", "'centerview", "'cf", "'china", "'clan", "'clothes", "'collector", "'comeback'i", "'coyote", "'craig", "'d.c.rip", "'dark", "'deliverance", "'diary", "'did", "'disney", "'distillation", "'dressing", "'duh", "'eject", "'em.anyways", "'error", "'estate", "'even", "'eye", "'fahreneheit", "'fahrenheit", "'fame", "'fillers'.an", "'flashdance", "'flashdance'and", "'forlane", "'foundation", "'fsol", "'funny", "'galang", "'gladiator", "'go", "'golden", "'heart", "'hell", "'helter", "'herland", "'highwayman", "'holt", "'how", "'i", "'if", "'incesticide", "'inferno", "'it", "'japanese", "'kuch", "'lab", "

              precision    recall  f1-score   support

           1       0.85      0.53      0.66      1987
           2       0.26      0.64      0.37       513

    accuracy                           0.56      2500
   macro avg       0.56      0.59      0.51      2500
weighted avg       0.73      0.56      0.60      2500



In [15]:
sw = noise + list(high_tokens) + list(low_tokens)

vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'`living", "'accelerator", "'adoption", "'anno", "'arular", "'ascension", "'ashes", "'away", "'bertha", "'big", "'brave", "'bubbabrain", "'bubbabyte", "'bucky", "'buffy", "'cade", "'canon", "'catch", "'centerview", "'cf", "'china", "'clan", "'clothes", "'collector", "'comeback'i", "'coyote", "'craig", "'d.c.rip", "'dark", "'deliverance", "'diary", "'did", "'disney", "'distillation", "'dressing", "'duh", "'eject", "'em.anyways", "'error", "'estate", "'even", "'eye", "'fahreneheit", "'fahrenheit", "'fame", "'fillers'.an", "'flashdance", "'flashdance'and", "'forlane", "'foundation", "'fsol", "'funny", "'galang", "'gladiator", "'go", "'golden", "'heart", "'hell", "'helter", "'herland", "'highwayman", "'holt", "'how", "'i", "'if", "'incesticide", "'inferno", "'it", "'japanese", "'kuch", "'lab", "

              precision    recall  f1-score   support

           1       0.48      0.62      0.54       970
           2       0.70      0.58      0.63      1530

    accuracy                           0.59      2500
   macro avg       0.59      0.60      0.59      2500
weighted avg       0.62      0.59      0.60      2500



In [16]:
sw = noise + list(high_tokens) + list(med_tokens)

vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
bow = vec.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vec.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'ve", '``', 'could', 'might', 'must', 'need', 'sha', 'wo', 'would'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           1       0.83      0.85      0.84      1212
           2       0.85      0.83      0.84      1288

    accuracy                           0.84      2500
   macro avg       0.84      0.84      0.84      2500
weighted avg       0.84      0.84      0.84      2500



Сравните полученные результаты, оцените какие токены наиболее важные для классификации.  
  
Лучшие показатели продемонстрировали токены с низкой частотой, но надо учитывать, что токены с низкой частотой были выбраны вручную. В целом "черные лебеди" более непредсказуемы, значит их последствия могут быть более значимы, а значит и токены с низкой частотой могут иметь схожее положение.

### Задание 2.  

Найти фичи с наибольшей значимостью, и вывести их

In [17]:
f = []
conv_to_int = [2 if el == '2' else 1 for el in y_test]
for i in tokens:
    cool_token = i
    pred = [2 if cool_token in sense else 1 for sense in x_test]
    f.append((i,accuracy_score(pred, conv_to_int)))
print(sorted(f)[:10])

[("''", 0.4968), ("''", 0.4968), ("''", 0.4968), ("''", 0.4968), ("''", 0.4968), ("''", 0.4968), ("''", 0.4968), ("''", 0.4968), ("''", 0.4968), ("''", 0.4968)]


In [20]:
print(sorted(f, reverse=True)[:10])
print(sorted(set(f))[:10])

[('éviter', 0.4964), ('étre', 0.4968), ('étai', 0.4968), ('émouvantes', 0.4972), ('è', 0.4972), ('è', 0.4972), ('è', 0.4972), ('è', 0.4972), ('à', 0.4964), ('à', 0.4964)]
[("''", 0.4968), ("'0", 0.4972), ("'01", 0.4968), ("'02", 0.4968), ("'100", 0.4968), ("'16", 0.4968), ("'1984", 0.4972), ("'50", 0.4972), ("'50's-'70", 0.4972), ("'50s", 0.4968)]


### Задание 3.
1) сравнить count/tf-idf/hashing векторайзеры/полносвязанную сетку (построить classification_report)

2) подобрать оптимальный размер для hashing векторайзера

3) убедиться что для сетки нет переобучения

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

In [31]:
c_filter = stopwords.words('english') + list(punctuation)

count_vect = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=c_filter)
bow = count_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(count_vect.transform(x_test))
print(classification_report(pred, y_test))

# vec = CountVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words=sw)
# bow = vec.fit_transform(x_train)
# clf = LogisticRegression(random_state=42)
# clf.fit(bow, y_train)
# pred = clf.predict(vec.transform(x_test))
# print(classification_report(pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", '``', 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  % sorted(inconsistent)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


              precision    recall  f1-score   support

           1       0.86      0.85      0.86      1255
           2       0.85      0.86      0.86      1245

    accuracy                           0.86      2500
   macro avg       0.86      0.86      0.86      2500
weighted avg       0.86      0.86      0.86      2500



In [32]:
o_filter = stopwords.words('english')
tf_vect = TfidfVectorizer(ngram_range=(1, 1), tokenizer=word_tokenize, stop_words= o_filter)
bow = tf_vect.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(tf_vect.transform(x_test))
print(classification_report(pred, y_test))

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  % sorted(inconsistent)


              precision    recall  f1-score   support

           1       0.88      0.86      0.87      1267
           2       0.86      0.87      0.87      1233

    accuracy                           0.87      2500
   macro avg       0.87      0.87      0.87      2500
weighted avg       0.87      0.87      0.87      2500



In [33]:
vectorizer = HashingVectorizer(n_features=2**4,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           1       0.63      0.57      0.60      1370
           2       0.53      0.59      0.56      1130

    accuracy                           0.58      2500
   macro avg       0.58      0.58      0.58      2500
weighted avg       0.59      0.58      0.58      2500



In [34]:
vectorizer = HashingVectorizer(n_features=2**8,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           1       0.75      0.74      0.75      1277
           2       0.73      0.75      0.74      1223

    accuracy                           0.74      2500
   macro avg       0.74      0.74      0.74      2500
weighted avg       0.74      0.74      0.74      2500



In [35]:
vectorizer = HashingVectorizer(n_features=2**10,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           1       0.83      0.80      0.81      1284
           2       0.80      0.82      0.81      1216

    accuracy                           0.81      2500
   macro avg       0.81      0.81      0.81      2500
weighted avg       0.81      0.81      0.81      2500



In [36]:
vectorizer = HashingVectorizer(n_features=2**12,)
bow = vectorizer.fit_transform(x_train)
clf = LogisticRegression(random_state=42)
clf.fit(bow, y_train)
pred = clf.predict(vectorizer.transform(x_test))
print(classification_report(pred, y_test))

              precision    recall  f1-score   support

           1       0.86      0.83      0.84      1292
           2       0.82      0.85      0.84      1208

    accuracy                           0.84      2500
   macro avg       0.84      0.84      0.84      2500
weighted avg       0.84      0.84      0.84      2500

